In [ ]:
%load_ext autoreload
%autoreload 2
# "%matplotlib widget" is slightly better, but sometimes doesn't work
# "%matplotlib notebook" or "%matplotlib inline" can be used as alternatives
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
# Debug: show location of imported remote_control code
import remote_control
remote_control.__file__

# Configure acquisition area

In [ ]:
from remote_control import acquisition
from remote_control.control import configure_fly_at_fixed_z
CONFIG_FN = 'remote_config.json'
slides = acquisition.SLIDES
#print(slides)

custom_plate =  { 
    "name": "labtek8",
    "spot_spacing": (10.5, 12.5, 1), #hor,ver (mm) centre to centre distance
    "spot_size": (4.5 ,7, 1), #h, v (mm)
    "grid_size": (2, 4), # h, v,
    "shape": "rectangle",
}

ga = acquisition.WellPlateGridAquisition(
    plate_type=custom_plate, 
    config_fn=CONFIG_FN, 
)

instrument_positions = [
         (422, 4252, -4.4),
         (-10252, 4187, -94.5),
         (29, 42684, 122.5),
         (-10536, 42666, 43)
]
#wells for calibration
wells = [
    (0,0,0),
    (1,0,0),
    (0,3,0),
    (1,3,0),
] 
ga.calibrate(instrument_positions, wells, 'centre') #calibration point location!

# Set up safety bounds (optional - comment this out if they're unwanted)
ga.set_image_bounds(
    min_x=-13900,
    max_x=3700,
    min_y=-450,
    max_y=48000,
)

# For plates with recessed wells, uncomment and configure this to move the slide away when moving between wells.
# If the stage needs to move in the X/Y plane more than "distance", it will move the stage's Z axis
# to the value of the "z" parameter. 
# configure_fly_at_fixed_z(distance=2000, z=3000) # Enable
configure_fly_at_fixed_z(distance=None, z=None) # Disable

spotsToUse = [
(0, 0),
(1, 0),
#(0, 1),
(1, 1),
#(0, 2), 
(1, 2),
(0, 3),
# (1, 3),
]

# Generate coordinates & preview
ga.generate_targets(
    spotsToUse, 
    offset_x=0, 
    offset_y=0, 
    pixelsize_x=50, 
    pixelsize_y=50, 
    area_function_name='lower_right')
ga.plot_targets(annotate=True)

# Write coordinates to file

In [ ]:
OUTPUT_DIR = 'D:\\imagingMS\\2021_08\\your name\\'
IMZML_PREFIX = OUTPUT_DIR + '01052019_Mouse_DHB_pos_mz200-800_px50x50_LR'

In [ ]:
ga.write_imzc_coords_file(OUTPUT_DIR + 'positions_imzc.txt')
ga.write_json_coords_file(OUTPUT_DIR + 'positions.json')

# Run acquisition

In [ ]:
ga.acquire(
    filename=IMZML_PREFIX, # Prefix for output coordinates files
    dummy=True, # False - send commands to MALDI, True - don't connect, just print commands
    measure=False, # False - move stage only, True - move stage & acquire data
    email_on_success='test@example.com', # Set to None to suppress
    email_on_failure='test@example.com',  # Set to None to suppress
)

# Cleanup
After imaging run the following cell to terminate Telnet

In [ ]:
import remote_control.control as rc
rc.close(quit=True)